In [1]:
#Load the joined audit snapshot (or raw and join again)
import pandas as pd
df = pd.read_csv("data/processed/join_audit_snapshot.csv")
device = pd.read_csv("data/raw/device_events.csv")

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/join_audit_snapshot.csv'

In [ ]:
#parse dates
df["dob"] = pd.to_datetime(df["dob"], errors="coerce")
device["event_timestamp"] = pd.to_datetime(device["event_timestamp"], errors="coerce")

df["birth_year"] = df["dob"].dt.year
df["age_est"] = (pd.Timestamp("today").normalize() - df["dob"]).dt.days / 365.25
df[["dob","birth_year","age_est"]].head()

In [ ]:
#identify features from ssn_pattern and address 
df["ssn_prefix"] = df["ssn_pattern"].astype(str).str.split("-").str[0]
df["ssn_prefix"] = pd.to_numeric(df["ssn_prefix"], errors="coerce")

df["address_num"] = df["address"].astype(str).str.extract(r"^(\d+)").astype(float)
df["address_len"] = df["address"].astype(str).str.len()

In [ ]:
#device aggregation features (fraud signals)
dev_agg = (
    device.groupby("user_id")
    .agg(
        n_events=("event_timestamp","size"),
        n_unique_devices=("device_id","nunique"),
        n_unique_ips=("ip_address","nunique"),
        n_unique_geos=("geo_location","nunique"),
        first_event=("event_timestamp","min"),
        last_event=("event_timestamp","max"),
    )
    .reset_index()
)

dev_agg["active_span_hours"] = (dev_agg["last_event"] - dev_agg["first_event"]).dt.total_seconds() / 3600.0
dev_agg["events_per_hour"] = dev_agg["n_events"] / (dev_agg["active_span_hours"].replace(0, np.nan))
dev_agg["events_per_hour"] = dev_agg["events_per_hour"].fillna(dev_agg["n_events"])  # if span 0

df = df.merge(dev_agg, on="user_id", how="left")
df[["n_events","n_unique_devices","n_unique_ips","n_unique_geos","events_per_hour"]].head()

In [ ]:
#metadata features
df["email_age_days"] = df["email_age_days"].fillna(df["email_age_days"].median())
df["disposable_email_flag"] = df["disposable_email_flag"].fillna(0).astype(int)

# Carrier frequency as a risk proxy
carrier_freq = df["phone_carrier"].value_counts(normalize=True)
df["phone_carrier_freq"] = df["phone_carrier"].map(carrier_freq).fillna(0)

In [ ]:
#target and final feature set
#Convert multiclass label to binary fraud indicator for baseline model
#If you want multiclass later, keep fraud_label as-is.
df["is_fraud"] = (df["fraud_label"] != 0).astype(int)

feature_cols = [
    "age_est","ssn_prefix","address_num","address_len",
    "email_age_days","disposable_email_flag","phone_carrier_freq",
    "n_events","n_unique_devices","n_unique_ips","n_unique_geos",
    "active_span_hours","events_per_hour"
]

X = df[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
y = df["is_fraud"]

X.head(), y.value_counts()

#save processed table
out = df[["user_id","fraud_label","is_fraud"] + feature_cols].copy()
out.to_csv("data/processed/training_table.csv", index=False)
out.head()


In [ ]:
#save processed table
out = df[["user_id","fraud_label","is_fraud"] + feature_cols].copy()
out.to_csv("data/processed/training_table.csv", index=False)
out.head()